In [27]:
import numpy as np
import sympy as sp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [28]:
"""
Create Augmented matrix from user input
"""

def create_matrix():
    # Get number of rows and columns from user input
    print("Enter the dimensions of the matrix:")
    rows = int(input("Enter the number of rows: "))
    cols = int(input("Enter the number of columns (including the augmented column): "))

    matrix = []  # Create an empty matrix

    # Populate the matrix with user input
    for i in range(rows):
        while True:
            row = input(f"Enter elements of the matrix at row {i+1} (including element at augmented column), separated by commas: ")
            row = list(map(int, row.split(',')))

            # Check if the row has the correct number of elements
            if len(row) != cols:
                print(f"Error: You must enter exactly {cols} elements.")
            else:
                matrix.append(row)
                break
    
    while True:
        print("\nCurrent Matrix:")
        for row in matrix:
            print(" ".join(map(str, row[:-1])) + " | " + str(row[-1]))
        
        edit = input("Would you like to edit any value? (yes/no): ").strip().lower()
        if edit == 'yes':
            row_num = int(input("Enter the row number to edit: ")) - 1
            col_num = int(input("Enter the column number to edit: ")) - 1
            new_value = int(input(f"Enter the new value for Row {row_num+1}, Column {col_num+1}: "))
            matrix[row_num][col_num] = new_value
        elif edit == 'no':
            break
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")
    
    return sp.Matrix(matrix)

In [29]:
def generate_vectors(rref_matrix, pivot_columns):
    # Check if the system is inconsistent
    if any(all(rref_matrix[i, j] == 0 for j in range(rref_matrix.shape[1] - 1)) and rref_matrix[i, -1] != 0 for i in range(rref_matrix.shape[0])):
        return []

    # Handle single row case
    if rref_matrix.shape[0] == 1:
        return [rref_matrix[:, :-1]]

    free_columns = [rref_matrix[:, i] for i in range(rref_matrix.shape[1]) if i not in pivot_columns]

    for i in range(len(free_columns) - 1):
        free_columns[i] = free_columns[i].applyfunc(lambda x: -x)

    for idx in range(len(free_columns) - 2):
        free_columns[idx][idx + 1, 0] = 1

    return free_columns

In [30]:
def describe_solution(vectors):
    print("\nThe solution spans the following vector(s):\n")
    for vector in vectors:
        print(vector)

In [31]:
# def plot_solution_set(vectors):
#     vector_count = len(vectors)

#     if vector_count == 2:
#         print("The solution set forms a line.\n")
#         plot_line(vectors)
#     elif vector_count == 3:
#         print("The solution set forms a plane.\n")
#         plot_plane(vectors)

#     describe_solution(vectors)

def plot_solution_set(vectors):
    vector_count = len(vectors)

    if vector_count == 0:
        print("The system has no solutions (it is inconsistent).\n")
        return

    if vector_count == 2:
        print("The solution set forms a line.\n")
        plot_line(vectors)
    elif vector_count == 3:
        print("The solution set forms a plane.\n")
        plot_plane(vectors)

    describe_solution(vectors)


In [32]:
def plot_line(vectors):
    arrays = [np.array(vector).astype(np.float64) for vector in vectors]
    direction_vector = arrays[0]
    translation_vector = arrays[1]

    t = np.linspace(-10, 10, 100)
    line_points = np.outer(t, direction_vector)

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=line_points[:, 0],
        y=line_points[:, 1],
        z=line_points[:, 2],
        mode='lines',
        name='Direction Line'
    ))

    translated_points = line_points + translation_vector
    fig.add_trace(go.Scatter3d(
        x=translated_points[:, 0],
        y=translated_points[:, 1],
        z=translated_points[:, 2],
        mode='lines',
        name='Translated Line'
    ))

    fig.show()


In [33]:
def plot_plane(vectors):
    arrays = [np.array(vector).astype(np.float64) for vector in vectors]
    x_vector, y_vector, translation_vector = arrays

    x_range = np.linspace(-10, 10, 20)
    y_range = np.linspace(-10, 10, 20)
    X, Y = np.meshgrid(x_range, y_range)
    Z = 2 * X + 3 * Y

    fig = go.Figure()

    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        name='Original Plane'
    ))

    translated_Z = Z + translation_vector[2]
    fig.add_trace(go.Surface(
        x=X, y=Y, z=translated_Z,
        colorscale='Blues',
        name='Translated Plane'
    ))

    fig.update_layout(scene=dict(aspectmode="cube"))
    fig.show()


In [157]:
def main():
    matrix = create_matrix()
    rref_matrix, pivot_columns = matrix.rref()
    vectors = generate_vectors(rref_matrix, pivot_columns)
    plot_solution_set(vectors)

if __name__ == "__main__":
    main()

Enter the dimensions of the matrix:
Error: You must enter exactly 4 elements.

Current Matrix:
1 1 1 | 1
1 1 1 | 2
2 1 1 | 1

The solution spans the following vector(s):

Matrix([[0], [1], [0]])


In [34]:
def test_unique_solution():
    input_matrix = sp.Matrix([
        [1, 1, 1, 6],
        [0, 1, -1, 1],
        [2, -1, 1, 7]
    ])
    rref_matrix, pivot_columns = input_matrix.rref()
    vectors = generate_vectors(rref_matrix, pivot_columns)
    plot_solution_set(vectors)

test_unique_solution()


The solution spans the following vector(s):

Matrix([[4], [3/2], [1/2]])


In [35]:
def test_infinite_solutions_line():
    input_matrix = sp.Matrix([
        [1, 2, -1, 4],
        [2, 4, -2, 8],
        [-1, -2, 1, -4]
    ])
    rref_matrix, pivot_columns = input_matrix.rref()
    vectors = generate_vectors(rref_matrix, pivot_columns)
    plot_solution_set(vectors)

test_infinite_solutions_line()

The solution set forms a plane.




The solution spans the following vector(s):

Matrix([[-2], [1], [0]])
Matrix([[1], [0], [0]])
Matrix([[4], [0], [0]])


In [36]:
def test_infinite_solutions_plane():
    input_matrix = sp.Matrix([
        [1, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 0, 1, 0]
    ])
    rref_matrix, pivot_columns = input_matrix.rref()
    vectors = generate_vectors(rref_matrix, pivot_columns)
    plot_solution_set(vectors)

test_infinite_solutions_plane()


The solution spans the following vector(s):

Matrix([[0], [0], [0]])


In [37]:
def test_no_solutions():
    input_matrix = sp.Matrix([
        [1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 4]
    ])
    rref_matrix, pivot_columns = input_matrix.rref()
    vectors = generate_vectors(rref_matrix, pivot_columns)
    plot_solution_set(vectors)

test_no_solutions()

The system has no solutions (it is inconsistent).



In [38]:
def test_zero_matrix():
    input_matrix = sp.Matrix([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])
    rref_matrix, pivot_columns = input_matrix.rref()
    vectors = generate_vectors(rref_matrix, pivot_columns)
    plot_solution_set(vectors)

test_zero_matrix()


The solution spans the following vector(s):

Matrix([[0], [1], [0]])
Matrix([[0], [0], [1]])
Matrix([[0], [0], [0]])
Matrix([[0], [0], [0]])


In [39]:
def test_single_row():
    input_matrix = sp.Matrix([
        [1, 2, 3, 4]
    ])
    rref_matrix, pivot_columns = input_matrix.rref()
    vectors = generate_vectors(rref_matrix, pivot_columns)
    plot_solution_set(vectors)

test_single_row()


The solution spans the following vector(s):

Matrix([[1, 2, 3]])


add more inputs and edit the code to allow implementation of these:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

allow them to add line by line
print 
allow them to edit
then allow them to chnage the position they want
while loop to keep to running
until they say done

Write code to insert '|' before the last element, (i-2 or i-1)
[3, 5, 6 | 5]
[3, 5, 6 | 5]
[3, 5, 6 | 5]

prompt user: augmented matrix
ask them to do the b

ask the user or show them different things:
For the result section: it should focus one: inconsistent, unique solutions, free variables
When it is unique we will show them the output so we wont need to plot
When it is inconsistent too
Do more research on the plotting plane, and the use cases

For the report
do a case where you can plot and show the different scenarios (1 fee variable, 2 free etc)
[1,2,3,4,5...9] -> good example
add when the solution doesnt exist the parallel lines
refer to the textbook for plane and translation 

plot both planes

good conclusion

talk about the transaltion

dont forget homo and non-homo